In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.style.use("seaborn-v0_8")
plt.rcParams["figure.figsize"] = (12,6)

In [ ]:
orders = pd.read_csv("Order_semicolon.csv", delimiter=";")
customers = pd.read_csv("Customer_semicolon.csv", delimiter=";")
campaigns = pd.read_csv("Campaign_semicolon.csv", delimiter=";")
campaign_queue = pd.read_csv("CampaignQueue_semicolon.csv", delimiter=";")

In [ ]:
orders["totalAmount"] = pd.to_numeric(orders["totalAmount"], errors="coerce")
orders["takeOutTimeInSeconds"] = pd.to_numeric(orders["takeOutTimeInSeconds"], errors="coerce")
orders["preparationTime"] = pd.to_numeric(orders["preparationTime"], errors="coerce")

In [ ]:
orders["tempo_retirada_min"] = orders["takeOutTimeInSeconds"].fillna(orders["preparationTime"]) / 60

In [ ]:
orders["createdAt"] = pd.to_datetime(orders["createdAt"], errors="coerce", dayfirst=True)

In [ ]:
ranking_empresas = (
    orders.groupby("companyId")
    .agg(
        pedidos=("id", "count"),
        mediana_tempo=("tempo_retirada_min", "median"),
        p90_tempo=("tempo_retirada_min", lambda x: x.quantile(0.9)),
        ticket_medio=("totalAmount", "mean")
    )
    .reset_index()
    .sort_values("mediana_tempo")
)

In [ ]:
ranking_plataformas = (
    orders.groupby("engineName")
    .agg(
        pedidos=("id", "count"),
        ticket_medio=("totalAmount", "mean"),
        mediana_tempo=("tempo_retirada_min", "median")
    )
    .reset_index()
    .sort_values("pedidos", ascending=False)
)

In [ ]:
pedidos_dia = (
    orders.groupby(orders["createdAt"].dt.date)
    .agg(
        pedidos=("id", "count"),
        receita_total=("totalAmount", "sum"),
        ticket_medio=("totalAmount", "mean")
    )
    .reset_index()
)

In [ ]:
orders["hour"] = orders["createdAt"].dt.hour
vendas_hora = (
    orders.groupby("hour")["totalAmount"]
    .sum()
    .reset_index()
)

In [ ]:
melhor_horario = vendas_hora.loc[vendas_hora["totalAmount"].idxmax()]

In [ ]:
print("=== Busca Gulosa (Horário) ===")
print(f"Horário mais lucrativo: {int(melhor_horario['hour'])}h")
print(f"Total de vendas: R$ {melhor_horario['totalAmount']:.2f}")

In [ ]:
sns.barplot(data=vendas_hora, x="hour", y="totalAmount", palette="Blues_d")
plt.axvline(x=melhor_horario["hour"], color="red", linestyle="--",
            label=f"Melhor hora (gulosa): {int(melhor_horario['hour'])}h")

In [ ]:
plt.legend()
plt.title("Vendas por Hora - Busca Gulosa escolhe o pico imediato")
plt.xlabel("Hora do dia")
plt.ylabel("Total de vendas (R$)")
plt.show()

In [ ]:
orders["weekday"] = orders["createdAt"].dt.day_name(locale="pt_BR")
vendas_semana = (
    orders.groupby("weekday")["totalAmount"]
    .sum()
    .reset_index()
)

In [ ]:
melhor_dia = vendas_semana.loc[vendas_semana["totalAmount"].idxmax()]

In [ ]:
print("\n=== Busca Gulosa (Dia da Semana) ===")
print(f"Dia mais lucrativo: {melhor_dia['weekday']}")
print(f"Total de vendas: R$ {melhor_dia['totalAmount']:.2f}")